<a href="https://colab.research.google.com/github/takky0330/NLP/blob/master/gpt_2_simple_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-

In [2]:
% tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow
tensorflow.__version__

'1.15.2'

In [4]:
! pip3 install gpt_2_simple

  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.1-cp36-none-any.whl size=23581 sha256=04fc988617b5e2a589717f132d8a3bc038a75f462b7c5ecdb2f6d7aeb5eb46c9
  Stored in directory: /root/.cache/pip/wheels/0c/f8/23/b53ce437504597edff76bf9c3b8de08ad716f74f6c6baaa91a
Successfully built gpt-2-simple


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
"""gpt2-simple.ipynb
Automatically generated by Colaboratory.
"""

# Commented out IPython magic to ensure Python compatibility.
# %tensorflow_version 1.x
# !pip3 install gpt-2-simple

import tensorflow
print(tensorflow.__version__)

import gpt_2_simple as gpt2
import os
import requests


1.15.2
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
### gpt-2-simpleのデフォルト（英語版）のモデルのダウンロード
#model_name = "124M"
#base_dir = "/content/drive/My Drive/Colab Notebooks/gpt2_learning"
#model_dir = os.path.join(base_dir,"models")
#if not os.path.isdir(os.path.join(model_dir, model_name)):
#	print(f"Downloading {model_name} model...")
#	gpt2.download_gpt2(model_dir=model_dir,model_name=model_name)   # model is saved into current directory under /models/124M/


In [7]:
model_name = "ja-117M"
#model_name = "gpt2ja-medium"
base_dir     = "/content/drive/My Drive/Colab Notebooks/gpt2_learning_ja"
model_dir = os.path.join(base_dir,"models")


In [8]:
import shutil

### 日本語のモジュールを展開
if os.path.isdir('gpt2-japanese/'):
    shutil.rmtree('gpt2-japanese/')
if os.path.isdir(model_dir):
    shutil.rmtree(model_dir)
if os.path.isdir(base_dir + '/gpt2-japanese/'):
    shutil.rmtree(base_dir + '/gpt2-japanese/')

! wget https://robotdiver.takky.org/rd/Colab/gpt2-japanese.zip -O gpt2-japanese.zip
! unzip -o gpt2-japanese.zip

new_path = shutil.move('./gpt2-japanese/', model_dir)

--2020-11-27 08:27:00--  https://robotdiver.takky.org/rd/Colab/gpt2-japanese.zip
Resolving robotdiver.takky.org (robotdiver.takky.org)... 13.70.40.33
Connecting to robotdiver.takky.org (robotdiver.takky.org)|13.70.40.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1122436 (1.1M) [application/zip]
Saving to: ‘gpt2-japanese.zip’

gpt2-japanese.zip   100%[===================>]   1.07M   924KB/s    in 1.2s    

2020-11-27 08:27:03 (924 KB/s) - ‘gpt2-japanese.zip’ saved [1122436/1122436]

Archive:  gpt2-japanese.zip
   creating: gpt2-japanese/
  inflating: gpt2-japanese/encoder.json  
  inflating: __MACOSX/gpt2-japanese/._encoder.json  
  inflating: gpt2-japanese/encoder.py  
  inflating: __MACOSX/gpt2-japanese/._encoder.py  
  inflating: gpt2-japanese/model.py  
  inflating: __MACOSX/gpt2-japanese/._model.py  
  inflating: gpt2-japanese/hparams.json  
  inflating: __MACOSX/gpt2-japanese/._hparams.json  
  inflating: gpt2-japanese/sampling.py  
  inflating: __MA

In [9]:
### 日本語のモデルのダウンロード （初回のみ必須）
! wget https://www.nama.ne.jp/models/ja-117M.tar.bz2 -O ja-117M.tar.bz2
! tar xvfj ja-117M.tar.bz2
## medium は使えない
#! wget https://www.nama.ne.jp/models/gpt2ja-medium.tar.bz2 -O gpt2ja-medium.tar.bz2
#! tar xvfj gpt2ja-medium.tar.bz2

shutil.copyfile(os.path.join(model_dir, "encoder.json"), "./ja-117M/encoder.json")
shutil.copyfile(os.path.join(model_dir, "hparams.json"), "./ja-117M/hparams.json")
shutil.copyfile(os.path.join(model_dir, "vocab.bpe"), "./ja-117M/vocab.bpe")

if os.path.isdir(os.path.join(base_dir,"models", model_name)):
    shutil.rmtree(os.path.join(base_dir,"models", model_name))
new_path = shutil.move(model_name + '/', model_dir)

--2020-11-27 08:27:17--  https://www.nama.ne.jp/models/ja-117M.tar.bz2
Resolving www.nama.ne.jp (www.nama.ne.jp)... 112.78.112.176
Connecting to www.nama.ne.jp (www.nama.ne.jp)|112.78.112.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 473889781 (452M) [application/x-bzip2]
Saving to: ‘ja-117M.tar.bz2’

ja-117M.tar.bz2     100%[===================>] 451.94M  2.96MB/s    in 3m 29s  

2020-11-27 08:30:47 (2.16 MB/s) - ‘ja-117M.tar.bz2’ saved [473889781/473889781]

ja-117M/
ja-117M/model-7353200.meta
ja-117M/model-7353200.index
ja-117M/stm.model
ja-117M/stm.vocab
ja-117M/checkpoint
ja-117M/model-7353200.data-00000-of-00001


In [10]:
file_name = os.path.join(base_dir,"shakespeare.txt")
if not os.path.isfile(file_name):
	url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
	data = requests.get(url)
	
	with open(file_name, 'w') as f:
		f.write(data.text)
    

In [ ]:
## 元のモデル（ja-117M）で generate() しようと思ったが… エラー！ と思ったが…  init を追加したら動いた
## しかも1回目は失敗するので、エラーの場合は2度実行する
sess = gpt2.start_tf_sess()
model_dir = os.path.join(base_dir,"models")
print(os.path.join(model_dir, model_name))
## 追加
init = tensorflow.global_variables_initializer()
sess.run(init)

prefix="　吾輩　は　猫　で　ある。名前　は　まだ　ない。もう　すぐ　お昼　だ　。　なぜ　英語　が　返って　くる　ん　だろう　？　？　"
##
try:
    gpt2.generate(sess, model_name=model_name, model_dir=model_dir, prefix=prefix)
except:
    sess = gpt2.start_tf_sess()
    init = tensorflow.global_variables_initializer()
    sess.run(init)
    gpt2.generate(sess, model_name=model_name, model_dir=model_dir, prefix=prefix)


In [ ]:
#### FineTuning

sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_name,
							model_dir=model_dir,
              model_name=model_name,
							checkpoint_dir=os.path.join(base_dir,"checkpoint"),
              steps=1000)   # steps is max number of training steps


In [11]:
! ln -s "/content/drive/My Drive/Colab Notebooks/gpt2_learning_ja/models/" ./models

In [12]:
! ls

drive  gpt2-japanese.zip  ja-117M.tar.bz2  __MACOSX  models  sample_data


In [13]:
cd models

/content/drive/My Drive/Colab Notebooks/gpt2_learning_ja/models


In [15]:
! pip3 install sentencepiece

     |████████████████████████████████| 1.1MB 20.6MB/s 


In [16]:
#! pip3 install sentencepiece
! python3 gpt2-generate.py --context="吾輩　は　猫　で　ある。名前　は　まだ　ない。もう　すぐ　お昼　だ　。　なぜ　英語　が　返って　くる　ん　だろう　？　？　" --num_generate 1



2020-11-27 08:36:48.326645: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-27 08:36:48.389413: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-27 08:36:48.390025: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2020-11-27 08:36:48.390337: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-11-27 08:36:48.602440: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-11-27 08:36:48.737929: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
202

In [ ]:
gpt2.generate(sess, checkpoint_dir=os.path.join(base_dir,"checkpoint"))


In [ ]:
gpt2.generate(sess, checkpoint_dir=os.path.join(base_dir,"checkpoint"), prefix="2015 年")
